In [152]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
import seaborn as sns

In [153]:
link="https://raw.githubusercontent.com/kb22/Heart-Disease-Prediction/refs/heads/master/dataset.csv"
df=pd.read_csv(link)

In [154]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [155]:
df.shape #total number of row and column

(303, 14)

In [156]:
df['target'].unique()

array([1, 0])

In [157]:
df.isna()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,False,False,False,False,False,False,False,False,False,False,False,False,False,False
299,False,False,False,False,False,False,False,False,False,False,False,False,False,False
300,False,False,False,False,False,False,False,False,False,False,False,False,False,False
301,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [158]:
features=df.drop(['target',], axis=1)
target=df['target']

X=features.values
y=target.values

In [159]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [160]:
y_train = pd.Series(y_train)
y_test = pd.Series(y_test)


In [161]:
# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [162]:
X_train=torch.tensor(X_train, dtype=torch.float32)
X_test=torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Convert to float32 and reshape to (N, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [163]:
class SimpleBinaryClassifier(nn.Module):
    def __init__(self):
        super(SimpleBinaryClassifier, self).__init__()
        self.linear = nn.Linear(13, 1)  # 13 input features, 1 output for binary classification

    def forward(self, x):
        return self.linear(x)

In [176]:
model = SimpleBinaryClassifier()

# Define the loss function and optimizer
loss_function = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.04)

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()  # Clear previous gradients
    logits = model(X_train)  # Forward pass

    # Double-check data types
    print(f"logits dtype: {logits.dtype}, y_train dtype: {y_train.dtype}")

    loss = loss_function(logits, y_train)  # Compute loss
    loss.backward()  # Backward pass
    optimizer.step()  # Update weights

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")


logits dtype: torch.float32, y_train dtype: torch.float32
Epoch 1/100, Loss: 0.7366
logits dtype: torch.float32, y_train dtype: torch.float32
Epoch 2/100, Loss: 0.6629
logits dtype: torch.float32, y_train dtype: torch.float32
Epoch 3/100, Loss: 0.6014
logits dtype: torch.float32, y_train dtype: torch.float32
Epoch 4/100, Loss: 0.5515
logits dtype: torch.float32, y_train dtype: torch.float32
Epoch 5/100, Loss: 0.5118
logits dtype: torch.float32, y_train dtype: torch.float32
Epoch 6/100, Loss: 0.4806
logits dtype: torch.float32, y_train dtype: torch.float32
Epoch 7/100, Loss: 0.4562
logits dtype: torch.float32, y_train dtype: torch.float32
Epoch 8/100, Loss: 0.4372
logits dtype: torch.float32, y_train dtype: torch.float32
Epoch 9/100, Loss: 0.4223
logits dtype: torch.float32, y_train dtype: torch.float32
Epoch 10/100, Loss: 0.4106
logits dtype: torch.float32, y_train dtype: torch.float32
Epoch 11/100, Loss: 0.4014
logits dtype: torch.float32, y_train dtype: torch.float32
Epoch 12/100, Lo

In [177]:
with torch.no_grad():
    model.eval()
    test_logits = model(X_test)
    predictions = torch.sigmoid(test_logits) >= 0.5
    predicted_labels = (predictions > 0.5).float()
    accuracy = (predictions == y_test).sum().item() / y_test.size(0)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 86.84%


In [178]:
print("Actual vs Predicted Targets:")
for actual, predicted in zip(y_test[:10], predicted_labels[:10]):  # Print first 10 results
    print(f"Actual: {actual.item()} - Predicted: {predicted.item()}")

Actual vs Predicted Targets:
Actual: 1.0 - Predicted: 1.0
Actual: 0.0 - Predicted: 0.0
Actual: 0.0 - Predicted: 0.0
Actual: 0.0 - Predicted: 0.0
Actual: 1.0 - Predicted: 1.0
Actual: 0.0 - Predicted: 0.0
Actual: 1.0 - Predicted: 1.0
Actual: 1.0 - Predicted: 1.0
Actual: 1.0 - Predicted: 1.0
Actual: 0.0 - Predicted: 0.0
